In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K

In [ ]:
import tensorflow as tf
tf.__version__
tf.test.gpu_device_name()

In [ ]:
K.clear_session()

data_entrenamiento = './data/train'
data_validacion = './data/validation'

In [ ]:
epocas = 50 #numero de veces q se va a iterar sobre el set de datos.
width_shape = 224
height_shape = 224 # tamano al cual vamos a procesar las imagenes
batch_size = 64 #cantidad de imagenes que enviamos a procesar en cada uno de los pasos
clases=2 #tipo de imagenes q vamos a enviar
lr=0.0005 #determina el tamano de los ajustes que ara nuestra red neuronal

In [ ]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    class_mode='categorical')
validation_generator = valid_datagen.flow_from_directory(
    data_validacion,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
print('#Entrenamiento de modelo VGG16')

nb_train_samples = 10682
nb_validation_samples = 3562

image_input = Input(shape=(width_shape, height_shape, 3))

model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

last_layer = model.output
out = Dense(clases, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)



custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

custom_vgg_model.summary()

In [ ]:
model_history = custom_vgg_model.fit(
    train_generator,
    epochs=epocas,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)

In [ ]:
custom_vgg_model.save("./modelos/model_VGG16.h5")

In [ ]:
def plotTraining(hist, epochs, typeData):
    if typeData == "loss":
        plt.figure(1, figsize=(10, 5))
        yc = hist.history['loss']
        xc = range(epochs)
        plt.ylabel('Loss', fontsize=24)
        plt.plot(xc, yc, '-r', label='Loss Training')
    if typeData == "accuracy":
        plt.figure(2, figsize=(10, 5))
        yc = hist.history['accuracy']
        for i in range(0, len(yc)):
            yc[i] = 100 * yc[i]
        xc = range(epochs)
        plt.ylabel('Accuracy (%)', fontsize=24)
        plt.plot(xc, yc, '-r', label='Accuracy Training')
    if typeData == "val_loss":
        plt.figure(1, figsize=(10, 5))
        yc = hist.history['val_loss']
        xc = range(epochs)
        plt.ylabel('Loss', fontsize=24)
        plt.plot(xc, yc, '--b', label='Loss Validate')
    if typeData == "val_accuracy":
        plt.figure(2, figsize=(10, 5))
        yc = hist.history['val_accuracy']
        for i in range(0, len(yc)):
            yc[i] = 100 * yc[i]
        xc = range(epochs)
        plt.ylabel('Accuracy (%)', fontsize=24)
        plt.plot(xc, yc, '--b', label='Training Validate')

    plt.rc('xtick', labelsize=24)
    plt.rc('ytick', labelsize=24)
    plt.rc('legend', fontsize=18)
    plt.legend()
    plt.xlabel('Number of Epochs', fontsize=24)
    plt.grid(True)



In [ ]:
plotTraining(model_history,epocas,"loss")
plotTraining(model_history,epocas,"accuracy")
plotTraining(model_history,epocas,"val_loss")
plotTraining(model_history,epocas,"val_accuracy")

In [ ]:
import matplotlib.pyplot as plt
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
import numpy as np
import cv2

width_shape = 224
height_shape = 224

names = ['MELANOMA', 'NO_MELANOMA']

modelt = load_model("./modelos/model_VGG16.h5")
#modelt = custom_vgg_model

imaget_path = "nomelanoma3.jpg"
imaget=cv2.resize(cv2.imread(imaget_path), (width_shape, height_shape), interpolation = cv2.INTER_AREA)
xt = np.asarray(imaget)
xt=preprocess_input(xt)
xt = np.expand_dims(xt,axis=0)
preds = modelt.predict(xt)

print(names[np.argmax(preds)])


plt.imshow(cv2.cvtColor(np.asarray(imaget),cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [1]:
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, precision_score, recall_score, accuracy_score, roc_auc_score
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

width_shape = 224
height_shape = 224 # tamano al cual vamos a procesar las imagenes
batch_size = 3

names = ['MELANOMA','NO_MELANOMA']


test_data_dir = './data/test'

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(width_shape, height_shape),
    batch_size = batch_size,
    class_mode='categorical',
    shuffle=False)

custom_Model= load_model("./model/model_VGG16.h5")
#modelt = custom_model

predictions = custom_Model.predict_generator(generator=test_generator)

y_pred = np.argmax(predictions, axis=1)
y_real = test_generator.classes


matc=confusion_matrix(y_real, y_pred)

plot_confusion_matrix(conf_mat=matc, figsize=(7,7), class_names = names, show_normed=False)
plt.tight_layout()

print(metrics.classification_report(y_real,y_pred, digits = 4))

Found 3560 images belonging to 2 classes.


C:\Users\LENOVO LEGION Y545\AppData\Local\Temp\ipykernel_22564\2855043826.py:30: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = custom_Model.predict_generator(generator=test_generator)


AssertionError: len(class_names) should be equal to number ofclasses in the dataset